In [25]:
import yaml
stream = open('init.yaml')
rdata = yaml.load(stream)
#print rdata
for key, value in rdata.iteritems():
                    cmdlist = []
                    print key.strip('\r\n')
                    print 'ip: ' + value[0]['ip']
                    print 'login: ' + value[1]['login']
                    print 'password: ' + value[2]['password']
                    print 'enablepassword: ' + value[3]['enablepassword']
                    print 'sleep: ' + str(value[4]['sleep'])

IOU4
ip: 192.168.0.204
login: admin
password: cisco
enablepassword: cisco
sleep: 0
IOU3
ip: 192.168.0.203
login: admin
password: cisco
enablepassword: cisco
sleep: 0
